In [ ]:
import pandas as pd
import json
import re

# read the csv file into a dataframe
df = pd.read_csv('output_json.csv')
# subtable 1 for history
df_history = df


# delete all columns other than history
column_to_keep = 'history'
columns_to_delete = df.columns.difference([column_to_keep])
df_history.drop(columns=columns_to_delete, inplace=True)

# extract subject (summary of question), content of question, and date of post from history
df['history_parsed'] = df['history'].apply(lambda x: json.loads(x))

def extract_first_subject_and_content(history):
    if history:  # Check if the history is not empty
        first_item = history[0]  # Assuming the first item contains the relevant data
        subject = first_item.get('subject', None)
        content = first_item.get('content', 'Content not provided')
        date = first_item.get('created', None)
        return subject, content, date
    return None, 'Content not provided'
# Apply the function to each row and create new columns for 'subject' and 'content'
df_history['first_subject'], df_history['first_content'], df_history['first_date'] = zip(*df_history['history_parsed'].apply(extract_first_subject_and_content))

# delete the original history and history_parsed
column_to_keep = ['first_subject', 'first_content', 'first_date']
df_history = df_history.drop(columns=[col for col in df_history.columns if col not in column_to_keep])

# merging first_subject and first_content
df_history['Summary and Description'] = df_history['first_subject'].astype(str) + ' ' + df['first_content'].astype(str)

# delete first_subject and first_content
column_to_keep = ['first_date', 'Summary and Description']
df_history = df_history.drop(columns=[col for col in df_history.columns if col not in column_to_keep])

# In a separate file, we extracted the nr column the same way as history and saved it into a csv file
# subtable 2 for nr
df_date = pd.read_csv('output_nr.csv')
df_date.rename(columns={'nr': 'post number'}, inplace=True)

# merge df_history and df_date
final_df = pd.concat([df_history, df_date], axis=1)
final_df.rename(columns={'first_date': 'post date'}, inplace=True)

# removing html and javascript tags from QuestionAndDescription
def remove_html_tags(text):
    # Pattern to match HTML and script tags
    clean_text = re.sub('<.*?>', '', text)  # Remove HTML tags
    clean_text = re.sub('<script.*?</script>', '', clean_text, flags=re.DOTALL)  # Remove script tags
    return clean_text

final_df['Summary and Description'] = final_df['Summary and Description'].apply(remove_html_tags)

# convert the final csv file into a txt file
final_df.to_csv('finalData.txt', sep='\t', index=False)

In [51]:
final_df


,post date,Summary and Description,post number
0,2023-12-16T04:41:33Z,Thanks to all of you! I appreciate the Piazza ...,1747
1,2024-01-03T02:34:14Z,Resources &amp; Announcements Logistics\nImpor...,6
2,2023-08-24T03:56:54Z,"Search for Teammates! PA.load(""/dashboard/proj...",5
3,2023-12-14T21:49:04Z,Grades &amp; End of Term Announcements Hi ever...,1733
4,2023-11-30T19:53:17Z,Final Exam Instructions Please take the time t...,1423
...,...,...,...
1483,2023-08-31T05:41:54Z,Couple of Questions Regarding Computers in thi...,10
1484,2023-08-30T22:09:49Z,Challenge Exam Piazza I have registered for th...,9
1485,2023-08-27T03:09:18Z,Question Regarding the Format of the Lectures ...,8
1486,2023-08-26T12:38:46Z,Urgent question about cpsc110 waitlist As a ...,7
